# SUDOKU 問題（１）Google OR-Toold による解法

=============================================================================================

問題１６）以下の Google OR-Tools のコードを利用して、CSVファイルを読み込んで、解答の２次元配列（リスト）を関数を作成せよ。
https://github.com/google/or-tools/blob/master/examples/python/sudoku_sat.py

=============================================================================================

##### Copyright 2021 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


# sudoku_sat

<table align="left">
<td>
<a href="https://colab.research.google.com/github/google/or-tools/blob/master/examples/notebook/examples/sudoku_sat.ipynb"><img src="https://raw.githubusercontent.com/google/or-tools/master/tools/colab_32px.png"/>Run in Google Colab</a>
</td>
<td>
<a href="https://github.com/google/or-tools/blob/master/examples/python/sudoku_sat.py"><img src="https://raw.githubusercontent.com/google/or-tools/master/tools/github_32px.png"/>View source on GitHub</a>
</td>
</table>

First, you must install [ortools](https://pypi.org/project/ortools/) package in this colab.

In [ ]:
!pip install ortools

     |████████████████████████████████| 14.6 MB 21.7 MB/s 
     |████████████████████████████████| 1.1 MB 44.0 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# カレントディレクトリーを移動 --> 必須
%cd /content/drive/MyDrive/100本ノックチャレンジ/05_Optimization_100_knocks/SUDOKU/01_notebook
%ls -lah

/content/drive/MyDrive/100本ノックチャレンジ/05_Optimization_100_knocks/SUDOKU/01_notebook
total 136K
drwx------ 2 root root 4.0K Feb 10 14:13 data/
drwx------ 2 root root 4.0K Feb 10 14:13 .ipynb_checkpoints/
-rw------- 1 root root  21K Feb  9 16:01 L1-Sudoku.ipynb
drwx------ 2 root root 4.0K Feb 10 14:13 L1-Sudoku-master/
-rw------- 1 root root  25K Feb 13 04:32 ★Solver_LP_pulp.ipynb
-rw------- 1 root root 6.2K Feb  9 14:54 sudoku-aka.ipynb
-rw------- 1 root root 9.1K Feb  9 15:52 sudoku_cvxpy.ipynb
-rw------- 1 root root 3.3K Feb  9 15:36 SUDOKU_CVXPY.ipynb
-rw------- 1 root root  16K Feb  8 21:04 sudoku_OR-Tools_220204.ipynb
-rw------- 1 root root  29K Feb 13 04:33 ★sudoku_OR-Tools_220209.ipynb
-rw------- 1 root root  14K Feb  7 09:32 sudoku_sat_rev_220204.ipynb


In [ ]:
from ortools.sat.python import cp_model
import pandas as pd
import numpy as np

In [ ]:
# 二次元行列のSUDOKUデータを入れたら解く関数
def solve_sudoku(initial_grid ):
    """Solves the sudoku problem with the CP-SAT solver."""
    # Create the model.
    model = cp_model.CpModel()

    cell_size = 3
    line_size = cell_size**2
    line = list(range(0, line_size))
    cell = list(range(0, cell_size))

    grid = {}
    for i in line:
        for j in line:
            grid[(i, j)] = model.NewIntVar(1, line_size, 'grid %i %i' % (i, j))

    # AllDifferent on rows.
    for i in line:
        model.AddAllDifferent([grid[(i, j)] for j in line])

    # AllDifferent on columns.
    for j in line:
        model.AddAllDifferent([grid[(i, j)] for i in line])

    # AllDifferent on cells.
    for i in cell:
        for j in cell:
            one_cell = []
            for di in cell:
                for dj in cell:
                    one_cell.append(grid[(i * cell_size + di,
                                          j * cell_size + dj)])

            model.AddAllDifferent(one_cell)

    # Initial values.
    for i in line:
        for j in line:
            if initial_grid[i][j]:
                model.Add(grid[(i, j)] == initial_grid[i][j])

    # Solve and print out the solution.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    if status == cp_model.OPTIMAL:
        for i in line:
            print([int(solver.Value(grid[(i, j)])) for j in line])


In [ ]:
# initial_grid = [[],
#                 [], 
#                 [],
#                 [],
#                 [],
#                 [],
#                 [],
#                 [],
#                 []]

# solve_sudoku(initial_grid )

In [ ]:
initial_grid = [[0, 6, 0, 0, 5, 0, 0, 2, 0], 
                [0, 0, 0, 3, 0, 0, 0, 9, 0],
                [7, 0, 0, 6, 0, 0, 0, 1, 0], 
                [0, 0, 6, 0, 3, 0, 4, 0, 0],
                [0, 0, 4, 0, 7, 0, 1, 0, 0], 
                [0, 0, 5, 0, 9, 0, 8, 0, 0],
                [0, 4, 0, 0, 0, 1, 0, 0, 6], 
                [0, 3, 0, 0, 0, 8, 0, 0, 0],
                [0, 2, 0, 0, 4, 0, 0, 5, 0]]

df = pd.DataFrame(initial_grid)
df.to_csv("./data/b_001.csv", header=False, index=False)
df

,0,1,2,3,4,5,6,7,8
0,0,6,0,0,5,0,0,2,0
1,0,0,0,3,0,0,0,9,0
2,7,0,0,6,0,0,0,1,0
3,0,0,6,0,3,0,4,0,0
4,0,0,4,0,7,0,1,0,0
5,0,0,5,0,9,0,8,0,0
6,0,4,0,0,0,1,0,0,6
7,0,3,0,0,0,8,0,0,0
8,0,2,0,0,4,0,0,5,0


In [ ]:
df = pd.read_csv("./data/b_001.csv", header=None)
df

,0,1,2,3,4,5,6,7,8
0,0,6,0,0,5,0,0,2,0
1,0,0,0,3,0,0,0,9,0
2,7,0,0,6,0,0,0,1,0
3,0,0,6,0,3,0,4,0,0
4,0,0,4,0,7,0,1,0,0
5,0,0,5,0,9,0,8,0,0
6,0,4,0,0,0,1,0,0,6
7,0,3,0,0,0,8,0,0,0
8,0,2,0,0,4,0,0,5,0


In [ ]:
grid = np.array(df).tolist()
grid

[[0, 6, 0, 0, 5, 0, 0, 2, 0],
 [0, 0, 0, 3, 0, 0, 0, 9, 0],
 [7, 0, 0, 6, 0, 0, 0, 1, 0],
 [0, 0, 6, 0, 3, 0, 4, 0, 0],
 [0, 0, 4, 0, 7, 0, 1, 0, 0],
 [0, 0, 5, 0, 9, 0, 8, 0, 0],
 [0, 4, 0, 0, 0, 1, 0, 0, 6],
 [0, 3, 0, 0, 0, 8, 0, 0, 0],
 [0, 2, 0, 0, 4, 0, 0, 5, 0]]

In [ ]:
# CSVファイルのパスを入れたらSUDOKUデータを入れたら解く関数
def solve_sudoku_csv(filepath):
    """Solves the sudoku problem with the CP-SAT solver."""

    # データの準備
    df = pd.read_csv(filepath, header=None)
    initial_grid = np.array(df).tolist()

    # 問題の表示
    print("Problem")
    for l in initial_grid:
      print(l)
    print("\n")

    # Create the model.
    model = cp_model.CpModel()

    cell_size = 3
    line_size = cell_size**2
    line = list(range(0, line_size))
    cell = list(range(0, cell_size))

    grid = {}
    for i in line:
        for j in line:
            grid[(i, j)] = model.NewIntVar(1, line_size, 'grid %i %i' % (i, j))

    # AllDifferent on rows.
    for i in line:
        model.AddAllDifferent([grid[(i, j)] for j in line])

    # AllDifferent on columns.
    for j in line:
        model.AddAllDifferent([grid[(i, j)] for i in line])

    # AllDifferent on cells.
    for i in cell:
        for j in cell:
            one_cell = []
            for di in cell:
                for dj in cell:
                    one_cell.append(grid[(i * cell_size + di,
                                          j * cell_size + dj)])

            model.AddAllDifferent(one_cell)

    # Initial values.
    for i in line:
        for j in line:
            if initial_grid[i][j]:
                model.Add(grid[(i, j)] == initial_grid[i][j])

    # Solve and print out the solution.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL:
        print("Solution")
        for i in line:
            print([int(solver.Value(grid[(i, j)])) for j in line])

In [ ]:
%%time
solve_sudoku_csv("./data/b_001.csv")

Problem
[0, 6, 0, 0, 5, 0, 0, 2, 0]
[0, 0, 0, 3, 0, 0, 0, 9, 0]
[7, 0, 0, 6, 0, 0, 0, 1, 0]
[0, 0, 6, 0, 3, 0, 4, 0, 0]
[0, 0, 4, 0, 7, 0, 1, 0, 0]
[0, 0, 5, 0, 9, 0, 8, 0, 0]
[0, 4, 0, 0, 0, 1, 0, 0, 6]
[0, 3, 0, 0, 0, 8, 0, 0, 0]
[0, 2, 0, 0, 4, 0, 0, 5, 0]


Solution
[8, 6, 1, 4, 5, 9, 7, 2, 3]
[4, 5, 2, 3, 1, 7, 6, 9, 8]
[7, 9, 3, 6, 8, 2, 5, 1, 4]
[2, 1, 6, 8, 3, 5, 4, 7, 9]
[9, 8, 4, 2, 7, 6, 1, 3, 5]
[3, 7, 5, 1, 9, 4, 8, 6, 2]
[5, 4, 7, 9, 2, 1, 3, 8, 6]
[1, 3, 9, 5, 6, 8, 2, 4, 7]
[6, 2, 8, 7, 4, 3, 9, 5, 1]
CPU times: user 15.9 ms, sys: 5.09 ms, total: 21 ms
Wall time: 24.3 ms


In [ ]:
# http://www.sudokugame.org/
# 初級
initial_grid = [[0,0,0,2,8,0,6,7,0],
                [0,7,5,0,0,0,8,3,0], 
                [0,8,0,7,0,0,4,0,0],
                [2,0,7,5,0,9,1,6,8],
                [0,0,0,0,2,8,7,4,0],
                [9,1,8,0,7,6,0,0,3],
                [8,0,4,3,1,5,9,0,7],
                [7,5,0,8,0,2,0,1,6],
                [1,2,3,6,9,7,5,8,4]]
df = pd.DataFrame(initial_grid)
df.to_csv("./data/b_002.csv", header=False, index=False)

In [ ]:
# http://www.sudokugame.org/
# 最高級
initial_grid = [[0,3,0,0,0,8,0,0,0],
                [0,0,0,0,5,0,0,0,6], 
                [0,6,0,9,0,1,2,0,0],
                [0,0,7,0,0,6,0,0,4],
                [2,0,0,0,0,0,0,1,0],
                [0,0,9,3,0,0,0,0,5],
                [0,0,0,4,0,0,3,0,0],
                [0,4,0,6,0,9,5,0,0],
                [0,0,1,0,7,0,0,0,0]]
df = pd.DataFrame(initial_grid)
df.to_csv("./data/s_001.csv", header=False, index=False)

In [ ]:
%%time
solve_sudoku_csv("./data/b_002.csv")

Problem
[0, 0, 0, 2, 8, 0, 6, 7, 0]
[0, 7, 5, 0, 0, 0, 8, 3, 0]
[0, 8, 0, 7, 0, 0, 4, 0, 0]
[2, 0, 7, 5, 0, 9, 1, 6, 8]
[0, 0, 0, 0, 2, 8, 7, 4, 0]
[9, 1, 8, 0, 7, 6, 0, 0, 3]
[8, 0, 4, 3, 1, 5, 9, 0, 7]
[7, 5, 0, 8, 0, 2, 0, 1, 6]
[1, 2, 3, 6, 9, 7, 5, 8, 4]


Solution
[3, 9, 1, 2, 8, 4, 6, 7, 5]
[4, 7, 5, 9, 6, 1, 8, 3, 2]
[6, 8, 2, 7, 5, 3, 4, 9, 1]
[2, 4, 7, 5, 3, 9, 1, 6, 8]
[5, 3, 6, 1, 2, 8, 7, 4, 9]
[9, 1, 8, 4, 7, 6, 2, 5, 3]
[8, 6, 4, 3, 1, 5, 9, 2, 7]
[7, 5, 9, 8, 4, 2, 3, 1, 6]
[1, 2, 3, 6, 9, 7, 5, 8, 4]
CPU times: user 22.7 ms, sys: 3.42 ms, total: 26.1 ms
Wall time: 26.3 ms


In [ ]:
%%time
solve_sudoku_csv("./data/s_001.csv")

Problem
[0, 3, 0, 0, 0, 8, 0, 0, 0]
[0, 0, 0, 0, 5, 0, 0, 0, 6]
[0, 6, 0, 9, 0, 1, 2, 0, 0]
[0, 0, 7, 0, 0, 6, 0, 0, 4]
[2, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 9, 3, 0, 0, 0, 0, 5]
[0, 0, 0, 4, 0, 0, 3, 0, 0]
[0, 4, 0, 6, 0, 9, 5, 0, 0]
[0, 0, 1, 0, 7, 0, 0, 0, 0]


Solution
[1, 3, 5, 2, 6, 8, 4, 9, 7]
[9, 2, 8, 7, 5, 4, 1, 3, 6]
[7, 6, 4, 9, 3, 1, 2, 5, 8]
[3, 5, 7, 1, 9, 6, 8, 2, 4]
[2, 8, 6, 5, 4, 7, 9, 1, 3]
[4, 1, 9, 3, 8, 2, 7, 6, 5]
[6, 7, 2, 4, 1, 5, 3, 8, 9]
[8, 4, 3, 6, 2, 9, 5, 7, 1]
[5, 9, 1, 8, 7, 3, 6, 4, 2]
CPU times: user 26.7 ms, sys: 339 µs, total: 27.1 ms
Wall time: 31.1 ms


## 問題用CVSファイル作成

In [ ]:
initial_grid = [[],
                [], 
                [],
                [],
                [],
                [],
                [],
                [],
                []]

df = pd.DataFrame(initial_grid)
df.to_csv("./data/s_001.csv", header=False, index=False)

In [ ]:
# 中級
initial_grid = [[1,0,0,0,0,0,6,0,0],
                [0,8,0,0,6,9,0,0,0], 
                [5,0,0,0,0,0,0,4,1],
                [2,0,0,9,0,0,0,0,0],
                [0,0,8,0,0,3,0,0,0],
                [9,0,0,5,7,6,0,0,0],
                [0,0,0,0,0,0,0,0,0],
                [8,6,0,4,5,0,0,2,9],
                [0,0,7,0,0,2,0,0,8]]

df = pd.DataFrame(initial_grid)
df.to_csv("./data/m_001.csv", header=False, index=False)

In [ ]:
%%time
solve_sudoku_csv("./data/m_001.csv")

Problem
[1, 0, 0, 0, 0, 0, 6, 0, 0]
[0, 8, 0, 0, 6, 9, 0, 0, 0]
[5, 0, 0, 0, 0, 0, 0, 4, 1]
[2, 0, 0, 9, 0, 0, 0, 0, 0]
[0, 0, 8, 0, 0, 3, 0, 0, 0]
[9, 0, 0, 5, 7, 6, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[8, 6, 0, 4, 5, 0, 0, 2, 9]
[0, 0, 7, 0, 0, 2, 0, 0, 8]


Solution
[1, 2, 9, 3, 4, 5, 6, 8, 7]
[7, 8, 4, 1, 6, 9, 2, 3, 5]
[5, 3, 6, 7, 2, 8, 9, 4, 1]
[2, 1, 5, 9, 8, 4, 7, 6, 3]
[6, 7, 8, 2, 1, 3, 5, 9, 4]
[9, 4, 3, 5, 7, 6, 8, 1, 2]
[3, 5, 2, 8, 9, 1, 4, 7, 6]
[8, 6, 1, 4, 5, 7, 3, 2, 9]
[4, 9, 7, 6, 3, 2, 1, 5, 8]
CPU times: user 19.4 ms, sys: 2.73 ms, total: 22.2 ms
Wall time: 25.1 ms


In [ ]:
# 中級
initial_grid = [[9,0,0,0,4,0,0,3,0],
                [0,0,6,0,0,0,0,0,0], 
                [0,0,0,8,0,0,7,0,0],
                [0,0,0,0,0,0,0,0,0],
                [6,0,0,0,9,3,0,0,0],
                [0,0,0,0,0,0,8,0,5],
                [0,0,8,0,0,0,0,0,0],
                [0,7,5,1,0,0,0,0,0],
                [0,0,0,0,0,7,0,9,0]]

df = pd.DataFrame(initial_grid)
df.to_csv("./data/m_002.csv", header=False, index=False)

In [ ]:
%%time
solve_sudoku_csv("./data/m_002.csv")

Problem
[9, 0, 0, 0, 4, 0, 0, 3, 0]
[0, 0, 6, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 8, 0, 0, 7, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[6, 0, 0, 0, 9, 3, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 8, 0, 5]
[0, 0, 8, 0, 0, 0, 0, 0, 0]
[0, 7, 5, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 7, 0, 9, 0]


Solution
[9, 1, 7, 2, 4, 5, 6, 3, 8]
[8, 3, 6, 9, 7, 1, 2, 5, 4]
[5, 4, 2, 8, 3, 6, 7, 1, 9]
[7, 5, 1, 6, 2, 8, 9, 4, 3]
[6, 8, 4, 5, 9, 3, 1, 2, 7]
[3, 2, 9, 7, 1, 4, 8, 6, 5]
[1, 9, 8, 3, 5, 2, 4, 7, 6]
[4, 7, 5, 1, 6, 9, 3, 8, 2]
[2, 6, 3, 4, 8, 7, 5, 9, 1]
CPU times: user 21.2 ms, sys: 827 µs, total: 22 ms
Wall time: 23.2 ms


In [ ]:
# 上級
initial_grid = [[0,7,2,0,0,0,0,0,0],
                [0,0,0,1,0,0,0,0,4], 
                [0,0,0,0,0,0,0,0,0],
                [1,0,4,0,0,0,0,0,9],
                [0,0,0,0,6,7,3,0,0],
                [0,0,0,0,0,0,0,0,0],
                [8,0,0,9,0,0,0,0,0],
                [0,0,3,0,0,8,2,7,0],
                [0,0,0,0,0,0,0,6,0]]

df = pd.DataFrame(initial_grid)
df.to_csv("./data/u_001.csv", header=False, index=False)

In [ ]:
%%time
solve_sudoku_csv("./data/u_001.csv")

Problem
[0, 7, 2, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 4]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 4, 0, 0, 0, 0, 0, 9]
[0, 0, 0, 0, 6, 7, 3, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[8, 0, 0, 9, 0, 0, 0, 0, 0]
[0, 0, 3, 0, 0, 8, 2, 7, 0]
[0, 0, 0, 0, 0, 0, 0, 6, 0]


Solution
[4, 7, 2, 3, 8, 5, 1, 9, 6]
[3, 5, 6, 1, 7, 9, 8, 2, 4]
[9, 1, 8, 6, 2, 4, 5, 3, 7]
[1, 6, 4, 2, 5, 3, 7, 8, 9]
[2, 8, 9, 4, 6, 7, 3, 1, 5]
[7, 3, 5, 8, 9, 1, 6, 4, 2]
[8, 2, 7, 9, 1, 6, 4, 5, 3]
[6, 9, 3, 5, 4, 8, 2, 7, 1]
[5, 4, 1, 7, 3, 2, 9, 6, 8]
CPU times: user 18 ms, sys: 1.77 ms, total: 19.8 ms
Wall time: 22.5 ms


In [ ]:
# 上級
initial_grid = [[0,8,0,3,0,0,9,0,0],
                [0,0,6,0,2,7,0,0,0], 
                [9,0,3,0,0,0,0,0,7],
                [2,0,0,0,0,0,1,0,8],
                [0,7,0,0,0,1,0,4,0],
                [1,0,4,0,3,0,0,0,0],
                [8,0,2,0,0,5,0,0,0],
                [4,0,9,0,0,0,2,0,0],
                [0,0,0,4,0,0,0,5,0]]

df = pd.DataFrame(initial_grid)
df.to_csv("./data/u_002.csv", header=False, index=False)

In [ ]:
%%time
solve_sudoku_csv("./data/u_002.csv")

Problem
[0, 8, 0, 3, 0, 0, 9, 0, 0]
[0, 0, 6, 0, 2, 7, 0, 0, 0]
[9, 0, 3, 0, 0, 0, 0, 0, 7]
[2, 0, 0, 0, 0, 0, 1, 0, 8]
[0, 7, 0, 0, 0, 1, 0, 4, 0]
[1, 0, 4, 0, 3, 0, 0, 0, 0]
[8, 0, 2, 0, 0, 5, 0, 0, 0]
[4, 0, 9, 0, 0, 0, 2, 0, 0]
[0, 0, 0, 4, 0, 0, 0, 5, 0]


Solution
[7, 8, 1, 3, 4, 6, 9, 2, 5]
[5, 4, 6, 9, 2, 7, 3, 8, 1]
[9, 2, 3, 5, 1, 8, 4, 6, 7]
[2, 9, 5, 6, 7, 4, 1, 3, 8]
[3, 7, 8, 2, 5, 1, 6, 4, 9]
[1, 6, 4, 8, 3, 9, 5, 7, 2]
[8, 3, 2, 1, 6, 5, 7, 9, 4]
[4, 5, 9, 7, 8, 3, 2, 1, 6]
[6, 1, 7, 4, 9, 2, 8, 5, 3]
CPU times: user 16.2 ms, sys: 1.07 ms, total: 17.2 ms
Wall time: 20.1 ms


In [ ]:
# 達人級
initial_grid = [[5,0,6,0,3,0,0,0,0],
                [0,0,0,0,0,9,0,1,4], 
                [0,0,0,0,0,0,0,0,0],
                [0,1,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,8,9,0],
                [2,0,0,5,0,0,6,0,0],
                [8,0,0,0,0,0,5,0,0],
                [0,4,0,0,0,1,0,0,0],
                [0,0,0,0,0,0,0,0,2]]

df = pd.DataFrame(initial_grid)
df.to_csv("./data/s_002.csv", header=False, index=False)

In [ ]:
%%time
solve_sudoku_csv("./data/s_002.csv")

Problem
[5, 0, 6, 0, 3, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 9, 0, 1, 4]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 8, 9, 0]
[2, 0, 0, 5, 0, 0, 6, 0, 0]
[8, 0, 0, 0, 0, 0, 5, 0, 0]
[0, 4, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 2]


Solution
[5, 9, 6, 1, 3, 4, 2, 7, 8]
[7, 8, 2, 6, 5, 9, 3, 1, 4]
[1, 3, 4, 7, 8, 2, 9, 5, 6]
[6, 1, 8, 9, 7, 3, 4, 2, 5]
[4, 5, 3, 2, 1, 6, 8, 9, 7]
[2, 7, 9, 5, 4, 8, 6, 3, 1]
[8, 2, 1, 3, 6, 7, 5, 4, 9]
[9, 4, 5, 8, 2, 1, 7, 6, 3]
[3, 6, 7, 4, 9, 5, 1, 8, 2]
CPU times: user 20.2 ms, sys: 2.7 ms, total: 22.9 ms
Wall time: 24.9 ms
